In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
spark = SparkSession.builder.appName('733').getOrCreate()
sc = spark.sparkContext

In [ ]:
annual_df = spark.read.csv('../annual_compustat.csv', header=True, inferSchema=True).limit(1000)

In [ ]:
rea_df = annual_df.select('rea')
rea_df = rea_df.filter(rea_df.rea.isNotNull())

In [ ]:
df = rea_df.select(col('rea'), rea_df.rea.cast('float').alias('float_rea'))
rea_df = rea_df.where(rea_df.rea != 0)
rea_df.show()

In [ ]:
data = df.select('float_rea').collect()

In [ ]:
# percentage = float(raw_input('What percentage? '))
import random
import numpy

percentage = 5
k = len(data) * percentage // 100
indices = random.sample(range(len(data)), k)
new_list1 = [data[i] for i in indices]

In [ ]:
len(new_list1)

In [ ]:
new_list1[0]
rea_vector = [x[0] for x in new_list1]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.hist(rea_vector, bins=50)
plt.ylabel('count');

In [ ]:
annual_df.show()

In [ ]:
nullcounts = spark.read.csv('annual_compustat_null_count.csv', header=False)

In [ ]:
import csv

with open('annual_compustat_null_count.csv', 'r') as f:
  reader = csv.reader(f)
  your_list = list(reader)

print(your_list)

In [ ]:
null_count_list = your_list[0]

In [ ]:
null_count_list = [float(x) for x in null_count_list]

In [ ]:
good_columns = []
for i in range(0, len(null_count_list)):
    if null_count_list[i]==0:
        good_columns.append(i)
    

In [ ]:
good_columns

In [ ]:
columns_num = [3, 10, 14]
df2 = annual_df.select(*(annual_df.columns[i] for i in good_columns))
df2.show()

In [ ]:
some_dict = {}
for x in annual_df.columns:
    some_dict[x] = 0
# some_dict

In [ ]:
permuted_annual_df = annual_df.fillna(some_dict)

In [ ]:
permuted_annual_df.first()

In [ ]:
non_string_columns = [k for (k,v) in permuted_annual_dtypes if v != 'string']

In [ ]:
permuted_annual_df_no_strings = permuted_annual_df.select(*non_string_columns)

In [ ]:
feature_columns = [item for item in permuted_annual_df_no_strings.columns if item not in ['rea', 'features']]

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

data = permuted_annual_df_no_strings

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=feature_columns, outputCol="features")

final_df = assembler.transform(data)

In [ ]:
final_df.show()

In [ ]:
final_final_df = final_df.drop(*feature_columns)

In [ ]:
final_final_df.show()

In [ ]:
final_final_df = final_final_df.withColumn('label', final_final_df.rea)

In [ ]:
final_final_df.show()

In [ ]:
final_final_df.write.parquet("final_final_df.parquet")

In [ ]:
ml_df = sqlContext.read.parquet("final_final_df.parquet")

In [ ]:
ml_df.show()

In [ ]:
# Split the data into train and test
splits = final_final_df.randomSplit([0.6, 0.4], 12)
train = splits[0]
test = splits[1]

# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [1514, 1514, 1514, 2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=128, seed=1234)

In [ ]:
train.show()

In [ ]:
# train the model
model = trainer.fit(train)

In [ ]:
# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))